geotiff_land.py                                                     <br>
                                                                    <br>
 This code converts SoilMoistureNPD in the AMSR-2 NRT Land product  <br>
   in HDF-EOS5 format to GeoTiff, with the use of GIBS color scheme <br>
   for the variable.                                                <br>
                                                                    <br>
 Considering the need for the use of composite soil moisture imagery<br>
   for a region that extends more than a single AMSR file,          <br>
   this code provides a way to make a composite imagery for the     <br>
   the product. To make GeoTiff for a variable from a single file,  <br>
   simply comment out "files= glob(...)"                            <br>
   and uncomment "files=['<filename with path>'] instead            <br>
                                                                    <br>
 Input: AMSR-2 NRT land file(s) in HDFeos5 POINTS data              <br>
 Output: GeoTIFF file of SoilMoistureNPD, composited from multiple  <br>
          files or from a single file                               <br>
                                                                    <br>
 Required Python packages:                                          <br>
     numpy, h5py, pyproj, gdal and GHRC's GIBScolors_utils          <br>
                                                                    <br>
 Notes:                                                             <br>
   The AMSR-U/2 Land product data are resampled to a global cyclic  <br>
   EASE-Grid (1383 cols x 586 rows) with a nominal spacing of 25 km <br>
  (true at 30° S). The data HDF-EOS point data resulting grid is in <br>
   table format, rather than a grid that image processing programs  <br>
   easily visualize.  i.e. to store the data the table uses flexible<br>
   data type (numpy.void in Python) which has combined data types   <br>
   in the data not accessible directly by GDAL.open()               <br>
   So here h5py is used to access the original *.h5e files          <br>
                                                                    <br>
   User can set <useGIBScolors> to True/False to elect use of GIBS  <br>
   color scheme or not.                                             <br>
   In the case of useGIBScolors=True, <gibsMapDir> the location of  <br>
   GIBS color scheme files needs to be provided                     <br>
                                                                    <br>
 By Y. WU of AMSR team @GHRC, Sep 2021                              <br>

In [2]:

import os,sys
import numpy as np
#import h5py
from glob import glob
from osgeo import gdal, osr
from pyproj import Proj, transform
from ipynb.fs.full.GIBScolors_utils import *

#--- a GDAL data type list for AMSR products
GDTconv={'uint8':  gdal.GDT_Byte ,
         'int16':  gdal.GDT_Int16,
         'int32':  gdal.GDT_Int32,
         'uint16': gdal.GDT_UInt16,
         'uint32': gdal.GDT_UInt32,
         'float32':gdal.GDT_Float32,
         'float64':gdal.GDT_Float64,  }

#epsg ='3410' for EASE-global
EASEgPrj4='+proj=cea +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +a=6371228 +b=6371228 +units=m +no_defs '
OutPrj = osr.SpatialReference()
OutPrj.ImportFromProj4(EASEgPrj4)

#---------------------------------------
#- Specify file(s) to be processed     -
#---------------------------------------
#--single or multiple file(s) to be processed
#files = ['C:/Users/lwang/Documents/AMSR/data/AMSR_U2_L2_Land_R02_202104060655_D.he5']

#---Alternative way to process multiple files
files = glob('test_files/geotiff/AMSR_U2_L2_Land_B02_202208101855*')

#--- variable to make geotiff file
nband=1
vname = 'SoilMoistureNPD'

#--- Option to use GIBS color scheme
useGIBScolors = True
gibsMapDir='colormaps/' #<-- ends with '/' for colormap files parsing

#--- get data
for ii, h5file in enumerate(files):  
    f1 = h5py.File(h5file, 'r')
    fields = 'Combined NPD and SCA Output Fields'
    if(ii==0):
        data = f1['HDFEOS/POINTS/AMSR-2 Level 2 Land Data/Data/'+fields]
        tstr1= h5file.split('_')[-2]
    else:
        #---concatenate data from multiple files
        dd2 = f1['HDFEOS/POINTS/AMSR-2 Level 2 Land Data/Data/'+fields]
        data= np.concatenate([data, dd2])

#--- output filename, accounting for time range in case of multiple files 
tstr = h5file.split('_')[-2]
if(tstr != tstr1): tstr = tstr1+'-'+tstr  #

#--- only take data with good samples, get (lat,lon)
data = data[data['FlagCountGoodSamples']>0]
varArr = data[vname]
banddsc = vname + ' ('+str(data[vname].dtype)+')' 

Lon = data['Longitude']
Lat = data['Latitude']

#--- geotransfer info using EASE-global grid to be used in geotiff
#    get the coverage for the current data points
Mx, My = 1383, 586
Xmin, Xmax = -17334193.54, 17334193.54
Ymin, Ymax =  -7344784.83,  7344784.83
dX, dY = (Xmax-Xmin)/Mx, (Ymin-Ymax)/My #<--dY is neg

p = Proj(EASEgPrj4)
X,Y = p(Lon,Lat,inverse=False)

# get (x,y) at the cell center
Icol = ((X-Xmin)/dX).astype(int)
Jrow = ((Y-Ymax)/dY).astype(int)
x = Xmin + (Icol + 0.5)*dX
y = Ymax + (Jrow + 0.5)*dY

xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()
NX = int((xmax - xmin)/dX+.001)+1
NY = int((ymin - ymax)/dY+.001)+1
Icol = Icol-Icol.min()
Jrow = Jrow-Jrow.min()

geotransform = (xmin,dX,0,ymax,0,dY)

#--- convert current data points on to the grid
fillV = -9999.
bandArr=np.empty([NY,NX]) 
bandArr.fill(fillV)
for a, j, i in zip(varArr, Jrow, Icol):
    bandArr[j,i] = a

#--- set output data type
if(useGIBScolors): 
    dtype='uint8'
    GDT = GDTconv[dtype]
    banddsc = banddsc.split('(')[0]+'('+dtype+')'
else:
    dtype = str(varArr.dtype)
    GDT = GDTconv[dtype]

#-----------------------------------------------
#- set driver, projection, geotransformation   -
#-  for output geotiff                         -
#-----------------------------------------------
#<--- output geotiff file name
TIFFpath= 'test_files/'+vname+'_'+tstr+'.tif'
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(TIFFpath,
                       NX, NY,
                       nband,
                       GDT)

out_ds.SetProjection(OutPrj.ExportToWkt())
out_ds.SetGeoTransform(geotransform)

#---activate output band
outBand = out_ds.GetRasterBand(1)
notVal = fillV

#--- if opted, use GIBS colors to set colortable
#    and reset NoDataValue value and rescale data to range set in GIBS colormap
scaleFC = {}
if(useGIBScolors): 
    gibsColors = gibsScheme(gibsMapDir, 'Land', vname, scaleFC)
    if gibsColors:
        notVal = 255
        colors = colorTable(gibsColors, 255, scaleFC, verb=False)
        outBand.SetColorTable(colors)
        outBand.SetColorInterpretation(gdal.GCI_PaletteIndex)
        
        #---rescale to range set in GIBS colormap
        rng1 = scaleFC['old'][0:2]
        rng2 = scaleFC['new'][0:2]
        
        good = bandArr != fillV
        g = bandArr[good]
        
        fctr = (rng2[1]-rng2[0])/(rng1[1]-rng1[0])
        bandArr[good] = (g-rng1[0]) * fctr + rng2[0]
        bandArr[bandArr>np.max(rng2)] = np.max(rng2)
        bandArr[~good] = notVal

outBand.WriteArray(bandArr)
outBand.SetDescription(banddsc)
outBand.SetNoDataValue(notVal)
        
outBand = None
out_ds = None  
print('Save to',TIFFpath,'\n')

test_files/geotiff/AMSR_U2_L2_Land_B02_202208101855_A.he5
passed print
 Use A2Land_NPDSM_LL_D.xml for SoilMoistureNPD
 Made 241 color intervals for the range [0.0-0.6]
Save to test_files/SoilMoistureNPD_202208101855.tif 

